In [2]:
#imports necessarios
import pandas as pd
import numpy as np
import kagglehub
import shutil
import os

pd.set_option('display.max_columns', None)


/home/mateus/Documentos/coffee_insight_project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Executando o download dos arquivos e movendo para a pasta 'data/raw'

In [3]:
raw_data_path = '../data/raw/'

# Download latest version
path = kagglehub.dataset_download("navjotkaushal/coffee-sales-dataset")

# Listar os arquivos baixados
files = os.listdir(path)
print(f"Arquivos encontrados: {files}")

# Mover cada arquivo do cache para a nossa pasta data/raw
for file_name in files:
    src_file = os.path.join(path, file_name)
    dst_file = os.path.join(raw_data_path, file_name)
    
    # Move o arquivo (substitui se já existir)
    shutil.move(src_file, dst_file)
    print(f"Arquivo movido com sucesso para: {dst_file}")

print("\nIngestão concluída!")



Arquivos encontrados: []

Ingestão concluída!


In [4]:
file_path = '../data/raw/Coffe_sales.csv'
df = pd.read_csv(file_path)
# Limpeza nos nomes das colunas deixando tudo minusculo
df.columns = [col.lower() for col in df.columns]
df.head(10)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3547 entries, 0 to 3546
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   hour_of_day  3547 non-null   int64  
 1   cash_type    3547 non-null   object 
 2   money        3547 non-null   float64
 3   coffee_name  3547 non-null   object 
 4   time_of_day  3547 non-null   object 
 5   weekday      3547 non-null   object 
 6   month_name   3547 non-null   object 
 7   weekdaysort  3547 non-null   int64  
 8   monthsort    3547 non-null   int64  
 9   date         3547 non-null   object 
 10  time         3547 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 304.9+ KB


In [ ]:
# 1. Informações gerais sobre colunas e tipos de dados
print("--- INFO ---")
df.info()

# 2. Contagem de valores nulos por coluna
print("\n--- VALORES NULOS ---")
print(df.isnull().sum())

# 3. Estatísticas básicas das colunas numéricas
print("\n--- DESCRIBE ---")
display(df.describe())


Diagnóstico:

1. O Problema da Coluna Date
Status atual: object 
Problema caso filtre por "vendas de janeiro" ou criar um gráfico de linha por dia vai tratar como palavras.

Solução: Converter para o tipo datetime64.

2. O Problema da Coluna Time
Status atual: object.

Problema: Mesma lógica da data. Para análises de "tempo de preparo" ou "picos de horário", precisamos que o Python entenda que isso representa horas e minutos.

Próxima Etapa: Tipagem e Padronização
Criar uma célula de Transformação para ajustar esses tipos. Isso garante que, quando subir para o banco de dados, as colunas já cheguem nos formatos corretos (DATE e TIME).

In [ ]:
# 2. Converter a data (formato simples)
df['date'] = pd.to_datetime(df['date'])

# 3. Converter o time usando 'mixed' para aceitar com e sem microssegundos
df['time'] = pd.to_datetime(df['time'], format='mixed').dt.time

# 4. Verificações finais de sucesso
print("--- Verificação Final ---")
print("Tipo do dado na coluna Time:", type(df['time'].iloc[0]))
print("Exemplo com microssegundos (Posição 0):", df['time'].iloc[0])
print("Exemplo sem microssegundos (Posição 484):", df['time'].iloc[484])

df.head()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3547 entries, 0 to 3546
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   hour_of_day  3547 non-null   int64         
 1   cash_type    3547 non-null   object        
 2   money        3547 non-null   float64       
 3   coffee_name  3547 non-null   object        
 4   time_of_day  3547 non-null   object        
 5   weekday      3547 non-null   object        
 6   month_name   3547 non-null   object        
 7   weekdaysort  3547 non-null   int64         
 8   monthsort    3547 non-null   int64         
 9   date         3547 non-null   datetime64[ns]
 10  time         3547 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 304.9+ KB


In [7]:
# Definir o caminho de destino na pasta 'processed'
output_path = "../data/processed/coffee_sales_clean.csv"

# Salvar o CSV
# index=False evita que o Pandas crie uma coluna extra de números sem necessidade
df.to_csv(output_path, index=False)

print(f"✅ Fase 1 Concluída! Arquivo salvo em: {output_path}")

✅ Fase 1 Concluída! Arquivo salvo em: ../data/processed/coffee_sales_clean.csv
